In [18]:
#source /scratch/phys/sin/sethih1/venv/MolNexTR_env/bin/activate
import sys
sys.path.append('/home/sethih1/MORAFInator/')

In [19]:
import os 
import sys
import time 
import json
import random
import argparse
import datetime
import numpy as np
import pandas as pd

import torch
import torch.distributed as dist
from torch.optim import Adam, AdamW, SGD
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from src.components import Encoder, Decoder
from src.loss_fuc import Criterion
from src.utils import seed_torch, save_args, init_summary_writer, LossMeter, AverageMeter, asMinutes, timeSince, print_rank_0, format_df
from src.chemical import convert_graph_to_smiles, postprocess_smiles, keep_main_molecule
from src.tokenization import get_tokenizer
#from src.dataloader.data_loader import get_datasets, afm_collate_fn

import warnings 
warnings.filterwarnings('ignore')

In [20]:
coord_bins = 64

coord_bin_values = np.round(np.linspace(0,1,64),2)
print(coord_bin_values)
coord_tokens = [f'<COORD_{i}>' for i in range(coord_bins)]

[0.   0.02 0.03 0.05 0.06 0.08 0.1  0.11 0.13 0.14 0.16 0.17 0.19 0.21
 0.22 0.24 0.25 0.27 0.29 0.3  0.32 0.33 0.35 0.37 0.38 0.4  0.41 0.43
 0.44 0.46 0.48 0.49 0.51 0.52 0.54 0.56 0.57 0.59 0.6  0.62 0.63 0.65
 0.67 0.68 0.7  0.71 0.73 0.75 0.76 0.78 0.79 0.81 0.83 0.84 0.86 0.87
 0.89 0.9  0.92 0.94 0.95 0.97 0.98 1.  ]


In [21]:

COVALENT_RADII = {
    1: 0.32,
    6: 0.77,
    7: 0.75,
    8: 0.73,
    9: 0.71,
}

ELEMENT_TO_INDEX = {
    1: 0,
    6: 1,
    7: 2,
    8: 3,
    9: 4,
}

INDEX_TO_SYMBOL = {
    1: 'H',
    6: 'C',
    7: 'N',
    8: 'O',
    9: 'F',
}



INDEX_TO_SYMBOL = {
    0: 'H',
    1: 'C',
    2: 'N',
    3: 'O',
    4: 'F',
}

In [23]:
from torch.utils.data import Dataset
import h5py

class AFMDataset(Dataset): 
    def __init__(self, data_path, tokenizer, transform, train_size=0.8, split='train'): 
        self.data_path = data_path
        self.transform = transform
        self.split = split
        self.tokenizer = tokenizer

        with h5py.File(self.data_path, 'r') as f:
            total_length = f['x'].shape[0]
            self.train_length = int(train_size*total_length)
            self.val_length = total_length - self.train_length

    def __len__(self): 
        if self.split == 'train': 
            return self.train_length
        else: 
            return self.val_length

    def __getitem__(self, idx):

        if self.split == 'train': 
            idx += 0
        else: 
            idx += self.train_length

        with h5py.File(self.data_path, 'r') as f:
            x = f['x'][idx]
            xyz = f['xyz'][idx]

        # Remove padding atoms
        xyz = xyz[xyz[:, -1] > 0]

        # Get edges 
        edges = []
        for i in range(xyz.shape[0]): 
            for j in range(i+1, xyz.shape[0]): 
                dist = np.linalg.norm(xyz[i, :3] - xyz[j, :3])
                if dist < 1.2*(COVALENT_RADII[xyz[i, -1]] + COVALENT_RADII[xyz[j, -1]]):
                    edges.append([i,j])

        # Normalize xyz to [0.25, 0.75]
        xyzmin = np.min(xyz[:, :3])
        xyz_max = np.max(xyz[:, 3])

        xyz[:, :3] = (xyz[:, :3] - xyzmin)/(xyz_max - xyzmin)
        xyz[:, :3] = 0.5*xyz[:, :3] + 0.25

        # map atom types to integers (0,1, ..)
        xyz[:, -1] = [ELEMENT_TO_INDEX[atom_type] for atom_type in xyz[:, -1]]

        #sample = {'coords': xyz, edges: np.asarray(edges)}

        #if self.transform:
        #    sample = self.transform(sample)

        # Keep all channels from HDF5 and convert to [C,H,W]
        x = torch.from_numpy(x)
        if x.dim() == 4 and x.size(0) == 1: 
            # Input like [1, H, W, C] -> [H, W, C]
            x = x.squeeze(0)
        if x.dim() == 3: 
            # Assume [H, W, C] from HDF5, move channels first
            x = x.permute(2,0,1).contiguous()

        mask = xyz[:, -1] > 0
        atomtok = " ".join([INDEX_TO_SYMBOL[value] for value in xyz[mask, -1]])

        #atomtok = self.tokenizer['atomtok'].text_to_sequence(atomtok)

        nodes = {'coords': [], 'symbols': ""}
        for symbol, coord in zip(atomtok, xyz):
            new_coord = coord_bin_values[np.argmin(np.abs(coord_bin_values.reshape(1,-1) - coord.reshape(-1,1)), axis = 1)]
            #atomtok_coords.append(f'{symbol}: {new_coord[0]}, {new_coord[1]}, {new_coord[2]},')
            nodes['coords'].append(new_coord[:2])
            nodes['symbols'] = symbol + " "

        atomtok_coords = self.tokenizer['atomtok_coords'].nodes_to_sequence(nodes)

        #ref = {'atomtok': atomtok, 'edges': np.asarray(edges), 'atomtok_coords': atomtok_coords, 'chartok_coords': atomtok_coords}
        #ref = {'atomtok_coords': atomtok_coords, 'atom_indices': , 'coords': nodes['coords'], 'edges': edges}
        ref = {'atomtok_coords': atomtok_coords}
        return idx, x, ref


def get_datasets(data_path, tokenizer, train_transform = None, val_transform = None, train_size=0.8):

    train_dataset = AFMDataset(data_path, tokenizer = tokenizer, transform=train_transform, train_size=train_size, split='train')
    val_dataset = AFMDataset(data_path, tokenizer = tokenizer, transform=val_transform, train_size=train_size, split='val')

    return train_dataset, val_dataset


def afm_collate_fn(batch):

    #sample = {'coords':[], 'edges':[]}
    ref = {'atomtok': [], 'edges': [], 'atomtok_coords': [], 'chartok_coords': []}
    ref = {'atomtok': []}
    #ref = {'atomtok_coords': []}

    PAD_ID = 0
    length = 128

    ids = [id[0] for id in batch]
    images = torch.stack([item[1] for item in batch])
    for item in batch:
        #sample['coords'].append(torch.from_numpy(item[2]['coords']))
        #sample['edges'].append(torch.from_numpy(item[2]['edges']))
        #ref['atomtok'].append(torch.from_numpy(item[2]['atomtok']))
        #ref['edges'].append(torch.from_numpy(item[2]['edges']))

        tok = item[2]['atomtok_coords']
        
        pad = [PAD_ID]*(length - len(tok))
        tok.extend(pad)
        
        ref['atomtok_coords'] = tok
        #ref['atomtok_coords'].append(torch.from_numpy(item[2]['atomtok_coords']))
        #ref['chartok_coords'].append(torch.from_numpy(item[2]['chartok_coords']))

    return ids, images, ref
    #return ids, images, sample

In [6]:
from types import SimpleNamespace

args_dict = {
    'batch_size': 32,
    'learning_rate': 0.001,

    # related to model encoder
    'encoder': 'swin_base', 
    'use_checkpoint':False, 
    'encoder_dim': 64, 
    'in_chans': 10,

    # related to model decoder
    'dec_hidden_size': 16,
    'enc_pos_emb': True, 
    'dec_num_layers': 3, 
    'dec_attn_heads': 4,
    'hidden_dropout': 0.2,
    'attn_dropout': 0.2,
    'max_relative_positions': 10,
    'compute_confidence': True,

    # related to tokenizer
    'formats':['atomtok_coords'],
    'vocab_file': '/home/sethih1/MORAFInator/src/vocab/vocab_chars.json', 
    'coord_bins': 64, 
    'sep_xy': False, 
    'continuous_coords': True, 
    'input_resolution': 256
    
}

args = SimpleNamespace(**args_dict)
print(args)

tokenizer = get_tokenizer(args)

namespace(batch_size=32, learning_rate=0.001, encoder='swin_base', use_checkpoint=False, encoder_dim=64, in_chans=10, dec_hidden_size=16, enc_pos_emb=True, dec_num_layers=3, dec_attn_heads=4, hidden_dropout=0.2, attn_dropout=0.2, max_relative_positions=10, compute_confidence=True, formats=['atomtok_coords'], vocab_file='/home/sethih1/MORAFInator/src/vocab/vocab_chars.json', coord_bins=64, sep_xy=False, continuous_coords=True, input_resolution=256)


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
print(device)

cuda


In [8]:
encoder = Encoder(args, pretrained=False).to(device)

In [9]:
from torchsummary import summary
#summary(encoder, input_size=(1, 10, 224, 224))

In [10]:
encoder_dim = encoder.n_features

In [11]:
decoder = Decoder(args, tokenizer)
print(decoder)

Decoder(
  (decoder): ModuleDict(
    (atomtok_coords): TransformerDecoderAR(
      (enc_trans_layer): Sequential(
        (0): Linear(in_features=64, out_features=16, bias=True)
      )
      (enc_pos_emb): Embedding(144, 64)
      (decoder): TransformerDecoder(
        (layer_norm): LayerNorm((16,), eps=1e-06, elementwise_affine=True)
        (transformer_layers): ModuleList(
          (0-2): 3 x TransformerDecoderLayer(
            (self_attn): MultiHeadedAttention(
              (linear_keys): Linear(in_features=16, out_features=16, bias=True)
              (linear_values): Linear(in_features=16, out_features=16, bias=True)
              (linear_query): Linear(in_features=16, out_features=16, bias=True)
              (softmax): Softmax(dim=-1)
              (dropout): Dropout(p=0.2, inplace=False)
              (final_linear): Linear(in_features=16, out_features=16, bias=True)
              (relative_positions_embeddings): Embedding(21, 4)
            )
            (feed_forward): 

In [12]:
encoder_optimizer = AdamW(encoder.parameters(), lr = 0.001, weight_decay=0.0001)
decoder_optimizer = AdamW(decoder.parameters(), lr= 0.001, weight_decay= 0.0005)

In [13]:
h5_path =  "/scratch/phys/project/sin/hackathon/data/afm.h5"
train_dataset, val_dataset = get_datasets(data_path = h5_path, tokenizer=tokenizer, train_transform = None, val_transform = None, train_size=0.8)

In [14]:
train_loader = DataLoader(train_dataset, batch_size=1, num_workers=1, drop_last = False, collate_fn=afm_collate_fn)

In [15]:
ids, imgs, samples = next(iter(train_loader))

In [16]:
samples['coords']


KeyError: 'coords'

In [17]:
# Training 

epochs = 10

for epoch in range(epochs):
    encoder.train()
    decoder.train()

    losses = []

    for step, (indices, imgs, refs) in enumerate(train_loader):

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        imgs = imgs.to(device)
        #refs = refs.to(device)


        features, hiddens = encoder(imgs)
        
        #features, hiddens = encoder(imgs, refs)
        results = decoder(features, hiddens, refs)
        losses = criterion(results, refs)
        loss = sum(losses.value())
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        losses.append(loss.value())

    train_loss = torch.mean(torch.tensor(losses))

    print(f"Epoch {epoch} training loss: {train_loss}")

ValueError: too many values to unpack (expected 2)

In [ ]:
data = create_3d_dataloader(h5_path, batch_size=1, num_workers=1)

In [ ]:


    def getitem(self, idx):
        ref = {}
        if self.dynamic_indigo:
            begin = time.time()
            image, smiles, graph, success = generate_indigo_image(
                self.smiles[idx], mol_augment=self.args.mol_augment, default_option=self.args.default_option,
                shuffle_nodes=self.args.shuffle_nodes, pseudo_coords=self.pseudo_coords,
                include_condensed=self.args.include_condensed)
            # raw_image = image
            end = time.time()
            if idx < 30 and self.args.save_image:
                path = os.path.join(self.args.save_path, 'images')
                os.makedirs(path, exist_ok=True)
                cv2.imwrite(os.path.join(path, f'{idx}.png'), image)
            if not success:
                return idx, None, {}
            image, coords = self.image_transform(image, graph['coords'], renormalize=self.pseudo_coords)
            graph['coords'] = coords
            ref['time'] = end - begin
            if 'atomtok' in self.formats:
                max_len = FORMAT_INFO['atomtok']['max_len']
                label = self.tokenizer['atomtok'].text_to_sequence(smiles, tokenized=False)
                ref['atomtok'] = torch.LongTensor(label[:max_len])
            if 'edges' in self.formats and 'atomtok_coords' not in self.formats and 'chartok_coords' not in self.formats:
                ref['edges'] = torch.tensor(graph['edges'])
            if 'atomtok_coords' in self.formats:
                self._process_atomtok_coords(idx, ref, smiles, graph['coords'], graph['edges'],
                                             mask_ratio=self.args.mask_ratio)
            if 'chartok_coords' in self.formats:
                self._process_chartok_coords(idx, ref, smiles, graph['coords'], graph['edges'],
                                             mask_ratio=self.args.mask_ratio)
            return idx, image, ref
        else:
            file_path = self.file_paths[idx]
            image = cv2.imread(file_path)
            if image is None:
                image = np.array([[[255., 255., 255.]] * 10] * 10).astype(np.float32)
                print(file_path, 'not found!')
            if self.coords_df is not None:
                h, w, _ = image.shape
                coords = np.array(eval(self.coords_df.loc[idx, 'node_coords']))
                if self.pseudo_coords:
                    coords = normalize_nodes(coords)
                coords[:, 0] = coords[:, 0] * w
                coords[:, 1] = coords[:, 1] * h
                image, coords = self.image_transform(image, coords, renormalize=self.pseudo_coords)
            else:
                image = self.image_transform(image)
                coords = None
            if self.labelled:
                smiles = self.smiles[idx]
                if 'atomtok' in self.formats:
                    max_len = FORMAT_INFO['atomtok']['max_len']
                    label = self.tokenizer['atomtok'].text_to_sequence(smiles, False)
                    ref['atomtok'] = torch.LongTensor(label[:max_len])
                if 'atomtok_coords' in self.formats:
                    if coords is not None:
                        self._process_atomtok_coords(idx, ref, smiles, coords, mask_ratio=0)
                    else:
                        self._process_atomtok_coords(idx, ref, smiles, mask_ratio=1)
                if 'chartok_coords' in self.formats:
                    if coords is not None:
                        self._process_chartok_coords(idx, ref, smiles, coords, mask_ratio=0)
                    else:
                        self._process_chartok_coords(idx, ref, smiles, mask_ratio=1)
            if self.args.predict_coords and ('atomtok_coords' in self.formats or 'chartok_coords' in self.formats):
                smiles = self.smiles[idx]
                if 'atomtok_coords' in self.formats:
                    self._process_atomtok_coords(idx, ref, smiles, mask_ratio=1)
                if 'chartok_coords' in self.formats:
                    self._process_chartok_coords(idx, ref, smiles, mask_ratio=1)
            return idx, image, ref

    def _process_atomtok_coords(self, idx, ref, smiles, coords=None, edges=None, mask_ratio=0):
        max_len = FORMAT_INFO['atomtok_coords']['max_len']
        tokenizer = self.tokenizer['atomtok_coords']
        if smiles is None or type(smiles) is not str:
            smiles = ""
        label, indices = tokenizer.smiles_to_sequence(smiles, coords, mask_ratio=mask_ratio)
        ref['atomtok_coords'] = torch.LongTensor(label[:max_len])
        indices = [i for i in indices if i < max_len]
        ref['atom_indices'] = torch.LongTensor(indices)
        if tokenizer.continuous_coords:
            if coords is not None:
                ref['coords'] = torch.tensor(coords)
            else:
                ref['coords'] = torch.ones(len(indices), 2) * -1.
        if edges is not None:
            ref['edges'] = torch.tensor(edges)[:len(indices), :len(indices)]
        else:
            if 'edges' in self.df.columns:
                edge_list = eval(self.df.loc[idx, 'edges'])
                n = len(indices)
                edges = torch.zeros((n, n), dtype=torch.long)
                for u, v, t in edge_list:
                    if u < n and v < n:
                        if t <= 4:
                            edges[u, v] = t
                            edges[v, u] = t
                        else:
                            edges[u, v] = t
                            edges[v, u] = 11 - t
                ref['edges'] = edges
            else:
                ref['edges'] = torch.ones(len(indices), len(indices), dtype=torch.long) * (-100)

    def _process_chartok_coords(self, idx, ref, smiles, coords=None, edges=None, mask_ratio=0):
        max_len = FORMAT_INFO['chartok_coords']['max_len']
        tokenizer = self.tokenizer['chartok_coords']
        if smiles is None or type(smiles) is not str:
            smiles = ""
        label, indices = tokenizer.smiles_to_sequence(smiles, coords, mask_ratio=mask_ratio)
        ref['chartok_coords'] = torch.LongTensor(label[:max_len])
        indices = [i for i in indices if i < max_len]
        ref['atom_indices'] = torch.LongTensor(indices)
        if tokenizer.continuous_coords:
            if coords is not None:
                ref['coords'] = torch.tensor(coords)
            else:
                ref['coords'] = torch.ones(len(indices), 2) * -1.
        if edges is not None:
            ref['edges'] = torch.tensor(edges)[:len(indices), :len(indices)]
        else:
            if 'edges' in self.df.columns:
                edge_list = eval(self.df.loc[idx, 'edges'])
                n = len(indices)
                edges = torch.zeros((n, n), dtype=torch.long)
                for u, v, t in edge_list:
                    if u < n and v < n:
                        if t <= 4:
                            edges[u, v] = t
                            edges[v, u] = t
                        else:
                            edges[u, v] = t
                            edges[v, u] = 11 - t
                ref['edges'] = edges
            else:
                ref['edges'] = torch.ones(len(indices), len(indices), dtype=torch.long) * (-100)


In [ ]:

# NEW: map element indices (from AFMData) to atomic symbols used by tokenizer
INDEX_TO_SYMBOL = {
    0: 'H',
    1: 'C',
    2: 'N',
    3: 'O',
    4: 'F',
}

class AFM3DToChartokConverter:
    """3D AFM to ChartTok converter for training"""
    
    def __init__(self, tokenizer, coord_bins=64, default_atom='C', use_3d=True):
        self.tokenizer = tokenizer
        self.coord_bins = coord_bins
        self.default_atom = default_atom
        self.use_3d = use_3d
        
        # Create coordinate tokens (bins for X, Y, Z)
        self.coord_tokens = [f'<COORD_{i}>' for i in range(coord_bins)]
    
    def coords_3d_to_token_sequence(self, coords_3d, symbols):
        """Convert 3D coordinates and symbols to token sequence (symbols + SOS/EOS)."""
        if coords_3d is None or len(coords_3d) == 0 or len(symbols) == 0:
            return torch.tensor([1, 2], dtype=torch.long)  # SOS, EOS
        
        sequence = [1]  # SOS token
        
        for symbol in symbols:
            if symbol in self.tokenizer.stoi:
                sequence.append(self.tokenizer.stoi[symbol])
            else:
                sequence.append(self.tokenizer.stoi.get(self.default_atom, UNK_ID))
        
        sequence.append(2)  # EOS
        return torch.tensor(sequence, dtype=torch.long)
    
    def _normalize_coords_3d(self, coords):
        """Normalize 3D coordinates to [0, 1] range"""
        coords = np.array(coords)
        if len(coords) == 0:
            return coords
        
        # Per-dimension normalization
        coords_norm = np.zeros_like(coords)
        for dim in range(3):
            coord_dim = coords[:, dim]
            min_val, max_val = coord_dim.min(), coord_dim.max()
            if max_val > min_val:
                coords_norm[:, dim] = (coord_dim - min_val) / (max_val - min_val)
            else:
                coords_norm[:, dim] = 0.5  # Default to middle if all same
        
        return coords_norm
    
    def convert_single_molecule_3d(self, nodes, edges=None):
        """Convert single AFM molecule to 3D chartok format"""
        if hasattr(nodes, 'x') and hasattr(nodes, 'pos'):
            # PyTorch Geometric format
            symbols = [self.default_atom] * len(nodes.x)  # Simplified - use default atom
            coords_3d = nodes.pos.numpy() if hasattr(nodes.pos, 'numpy') else nodes.pos
        elif isinstance(nodes, dict):
            # Dictionary format
            if 'pos' in nodes and 'x' in nodes:
                coords_3d = nodes['pos']
                symbols = [self.default_atom] * len(coords_3d)
            else:
                # Default fallback
                coords_3d = np.random.rand(5, 3)  # Random 3D positions
                symbols = [self.default_atom] * 5
        else:
            # Tensor or array format
            if hasattr(nodes, 'shape') and len(nodes.shape) >= 2:
                if nodes.shape[-1] >= 3:
                    coords_3d = nodes[:, :3] if len(nodes.shape) == 2 else nodes.reshape(-1, 3)[:, :3]
                    symbols = [self.default_atom] * len(coords_3d)
                else:
                    # Fallback to random
                    coords_3d = np.random.rand(5, 3)
                    symbols = [self.default_atom] * 5
            else:
                coords_3d = np.random.rand(5, 3)
                symbols = [self.default_atom] * 5
        
        # Convert to token sequence
        token_sequence = self.coords_3d_to_token_sequence(coords_3d, symbols)
        
        # Create info dict
        nodes_3d_dict = {
            'coords_3d': coords_3d.tolist() if hasattr(coords_3d, 'tolist') else coords_3d,
            'symbols': symbols,
            'num_atoms': len(symbols)
        }
        
        return token_sequence, nodes_3d_dict

# NEW: Dataset wrapper that uses AFMData (HDF5) and converts to model-friendly items
class H5AFM3DDataset(Dataset):
    def __init__(self, h5_path, tokenizer, coord_bins=64):
        super().__init__()
        self.afm = AFMData(h5_path, transform=None, train_size=1.0, split='train')
        self.converter = AFM3DToChartokConverter(tokenizer, coord_bins=coord_bins, default_atom='C', use_3d=True)
    def __len__(self):
        return len(self.afm)
    def __getitem__(self, idx):
        idx_out, image, sample = self.afm[idx]
        # sample['coords']: numpy array [num_atoms, 5] -> x,y,z,charge?,element_index(0..4)
        coords_np = sample['coords']
        edges_np = sample['edges']  # (E, 2)
        coords_3d = torch.from_numpy(coords_np[:, :3]).float() if coords_np.size > 0 else torch.zeros((0, 3), dtype=torch.float32)
        # derive symbols from last column (already mapped to 0..4)
        elem_idx = coords_np[:, -1].astype(int) if coords_np.size > 0 else np.array([], dtype=int)
        symbols = [INDEX_TO_SYMBOL.get(int(i), 'C') for i in elem_idx]
        # build token sequence for chartok_coords
        sequence = self.converter.coords_3d_to_token_sequence(coords_3d.numpy(), symbols)
        return {
            'idx': torch.tensor(idx_out, dtype=torch.long),
            'image': image,                         # tensor [3,H,W]
            'sequence': sequence,                   # tensor [T]
            'seq_length': torch.tensor(len(sequence), dtype=torch.long),
            'coords_3d': coords_3d,                 # tensor [N,3]
            'symbols': symbols,
            'num_atoms': torch.tensor(len(symbols), dtype=torch.long),
            'edges_list': torch.from_numpy(edges_np).long() if edges_np.size > 0 else torch.zeros((0, 2), dtype=torch.long),
        }

class Complete3DAFMDataset:
    """Complete 3D AFM dataset wrapper"""
    
    def __init__(self, afm_dataset, tokenizer, coord_bins=64, default_atom='C', use_3d=True):
        self.afm_dataset = afm_dataset
        self.converter = AFM3DToChartokConverter(tokenizer, coord_bins, default_atom, use_3d)
        self.cache = {}
    
    def __len__(self):
        return len(self.afm_dataset)
    
    def __getitem__(self, idx):
        if idx in self.cache:
            return self.cache[idx]
        
        # Get AFM sample
        afm_sample = self.afm_dataset[idx]
        
        # Extract molecular data
        nodes = afm_sample.get('nodes', afm_sample.get('x', None))
        edges = afm_sample.get('edges', None)
        
        if nodes is not None:
            chartok_3d_seq, nodes_3d_dict = self.converter.convert_single_molecule_3d(nodes, edges)
            
            converted_sample = {
                'idx': idx,
                'chartok_coords_3d': chartok_3d_seq,
                'nodes_3d_dict': nodes_3d_dict,
                'original_afm_sample': afm_sample,
                'coords_3d': torch.tensor(nodes_3d_dict['coords_3d'], dtype=torch.float32),
                'symbols': nodes_3d_dict['symbols']
            }
        else:
            # Fallback
            converted_sample = {
                'idx': idx,
                'chartok_coords_3d': torch.tensor([1, 2]),
                'nodes_3d_dict': {'coords_3d': [], 'symbols': []},
                'original_afm_sample': afm_sample,
                'coords_3d': torch.tensor([]),
                'symbols': []
            }
        
        self.cache[idx] = converted_sample
        return converted_sample

class AFM3DCollator:
    """Custom collate function for 3D AFM data"""
    
    def __init__(self, tokenizer, max_length=256):
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __call__(self, batch):
        batch_size = len(batch)
        
        sequences = [sample['chartok_coords_3d'] for sample in batch]
        coords_3d_list = [sample['coords_3d'] for sample in batch]
        symbols_list = [sample['symbols'] for sample in batch]
        
        # Pad sequences
        max_seq_len = min(max(len(seq) for seq in sequences), self.max_length)
        padded_sequences = torch.zeros((batch_size, max_seq_len), dtype=torch.long)
        attention_masks = torch.zeros((batch_size, max_seq_len), dtype=torch.bool)
        
        # 3D coordinates
        max_atoms = max(len(coords) for coords in coords_3d_list) if coords_3d_list[0].numel() > 0 else 1
        coords_3d_batch = torch.zeros((batch_size, max_atoms, 3))
        coords_mask = torch.zeros((batch_size, max_atoms), dtype=torch.bool)
        
        for i, (seq, coords_3d, symbols) in enumerate(zip(sequences, coords_3d_list, symbols_list)):
            seq_len = min(len(seq), max_seq_len)
            padded_sequences[i, :seq_len] = seq[:seq_len]
            attention_masks[i, :seq_len] = True
            
            if coords_3d.numel() > 0 and len(coords_3d.shape) == 2:
                atom_len = min(len(coords_3d), max_atoms)
                coords_3d_batch[i, :atom_len] = coords_3d[:atom_len]
                coords_mask[i, :atom_len] = True
        
        return {
            'input_ids': padded_sequences,
            'attention_mask': attention_masks,
            'coords_3d': coords_3d_batch,
            'coords_mask': coords_mask,
            'symbols_batch': symbols_list,
            'batch_size': batch_size,
            'max_seq_len': max_seq_len,
            'max_atoms': max_atoms
        }

def make_collate_fn(tokenizer):
    """Create a collate function that maps out-of-range tokens to UNK and batches coords/edges."""
    vocab_size = len(tokenizer)
    base_vocab = len(tokenizer.stoi)

    def _collate(batch):
        # indices and images
        indices = torch.stack([item['idx'] for item in batch])
        images = torch.stack([item['image'] for item in batch])

        # sequences with UNK clamping
        raw_sequences = [item['sequence'] for item in batch]
        sequences = []
        seq_lengths = []
        for seq in raw_sequences:
            seq = seq.clone()
            invalid = (seq >= vocab_size) | (seq < 0)
            if invalid.any():
                seq[invalid] = UNK_ID
            sequences.append(seq)
            seq_lengths.append(torch.tensor(len(seq), dtype=torch.long))
        seq_lengths = torch.stack(seq_lengths)

        # debug token range for first few batches
        all_tokens = torch.cat(sequences) if len(sequences) > 0 else torch.tensor([], dtype=torch.long)
        if all_tokens.numel() > 0:
            max_token = all_tokens.max().item(); min_token = all_tokens.min().item()
        else:
            max_token = min_token = 0
        if not hasattr(_collate, 'call_count'):
            _collate.call_count = 0
        _collate.call_count += 1
        if _collate.call_count <= 3:
            logger.info(f"🔍 Batch {_collate.call_count}: Token range [{min_token}, {max_token}], base_vocab={base_vocab}, total_vocab={vocab_size}, sequences={len(sequences)}")

        # pad sequences
        max_len = max(len(seq) for seq in sequences) if sequences else 0
        batch_size = len(batch)
        padded_sequences = torch.zeros(batch_size, max_len, dtype=torch.long)
        for i, seq in enumerate(sequences):
            L = len(seq)
            if L > 0:
                padded_sequences[i, :L] = seq

        # coords_3d batching (pad to max atoms), use -100 for masked positions
        coords_list = [item['coords_3d'] for item in batch]
        num_atoms_list = [int(item['num_atoms']) for item in batch]
        max_atoms = max(num_atoms_list) if num_atoms_list else 0
        coords_3d_batch = torch.full((batch_size, max_atoms, 3), -100.0, dtype=torch.float32)
        for i, coords in enumerate(coords_list):
            n = min(coords.size(0), max_atoms)
            if n > 0:
                coords_3d_batch[i, :n, :] = coords[:n, :]

        # atom_indices: positions of symbol tokens in sequence: [1..num_atoms]
        atom_indices = torch.zeros(batch_size, max_atoms, dtype=torch.long)
        for i, n_atoms in enumerate(num_atoms_list):
            n = min(n_atoms, max_atoms)
            if n > 0:
                atom_indices[i, :n] = torch.arange(1, 1 + n)

        # edges target placeholder: ignore all pairs by default (-100), shape [B, max_atoms, max_atoms]
        edges_target = torch.full((batch_size, max_atoms, max_atoms), -100, dtype=torch.long)

        refs = {
            'chartok_coords': (padded_sequences, seq_lengths),
            'coords_3d': coords_3d_batch,
            'edges': edges_target,
            'atom_indices': (atom_indices,),
        }
        return indices, images, refs

    return _collate

def create_3d_dataloader(data_path, batch_size=16, num_workers=4):
    """Create DataLoader for 3D AFM training from HDF5 using the provided pipeline"""
    args = create_model_args()

    # Determine HDF5 file path
    h5_path = data_path
    if os.path.isdir(data_path):
        # pick the first .h5 file in the directory
        h5_files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith('.h5') or f.endswith('.hdf5')]
        if len(h5_files) == 0:
            raise FileNotFoundError(f"No .h5/.hdf5 files found under {data_path}")
        h5_path = sorted(h5_files)[0]

    # Build tokenizer(s)
    tokenizers = get_tokenizer(args)
    chartok_tok = tokenizers['chartok_coords']
    logger.info(f"📚 Tokenizer built: chartok_coords | base_vocab={len(chartok_tok.stoi)} | total_vocab={len(chartok_tok)} | coord_bins={args.coord_bins}")

    # Create dataset from HDF5
    dataset = H5AFM3DDataset(h5_path, chartok_tok, coord_bins=args.coord_bins)
    logger.info(f"📂 Using AFM HDF5 dataset: {h5_path} | {len(dataset)} samples")

    # Collate function
    collate_fn = make_collate_fn(chartok_tok)

    train_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
        collate_fn=collate_fn
    )

    logger.info(f"✅ DataLoader created with {len(dataset)} samples, {len(train_loader)} batches")
    return train_loader, tokenizers, dataset
